# 下载数据并绘制雷达叠加自动站要素的综合图

### 第一步，下载数据，包括地面自动站小时观测数据、分钟级观测数据以及雷达单站基数据

In [1]:

# 下载自动站小时观测数据

from nmc_met_io.retrieve_cmadaas import cmadaas_obs_in_rect_by_time_range
import os
from datetime import datetime, timedelta
starttime = '20210720080000'
endtime = '20210720120000'
outpath='/mnt/e/metradar_test/radar_aws/awsdata/aws_hourly_obs'
if not os.path.exists(outpath):
    os.makedirs(outpath, exist_ok=True)
startt = datetime.strptime(starttime, '%Y%m%d%H%M%S')
endt = datetime.strptime(endtime, '%Y%m%d%H%M%S')
region = [33,36,112,115]
# 每次下载1小时的数据
curt = startt
while curt <= endt:
    nextt = curt + timedelta(hours=1)
    curtime = curt.strftime('%Y%m%d%H%M%S')
    nexttime = nextt.strftime('%Y%m%d%H%M%S')
    outname = f'{outpath}/surface_aws_hourly_{curtime[0:8]}_{curtime[8:12]}.csv'
    if os.path.exists(outname):
        print(f'文件已存在，跳过下载：{outname}')
        curt = nextt
        continue
    # 下载数据
    data = cmadaas_obs_in_rect_by_time_range( time_range=f'[{curtime},{nexttime}]',
                              limit=region,trans_type=False,
                              elements="Station_Id_C,Station_Id_d,Lat,Lon,Year,Mon,Day,Hour,\
                                  WIN_S_Avg_2mi,WIN_D_Avg_2mi,PRE_1h,Q_PRE_1h,WIN_S_Inst_Max,\
                                      WIN_D_INST_Max,WIN_S_INST_Max_OTime,PRS_Sea,TEM,RHU,DPT")
    # 保存到文件Z surface_aws_hourly_20250914_1000.csv
    # 去掉列名中的空格
    data.columns = [col.strip() for col in data.columns]
    data.to_csv(outname, index=False,encoding='GBK')
    print(f'已保存到文件：{outname}')

    
    curt = nextt

文件已存在，跳过下载：/mnt/e/metradar_test/radar_aws/awsdata/aws_hourly_obs/surface_aws_hourly_20210720_0800.csv
文件已存在，跳过下载：/mnt/e/metradar_test/radar_aws/awsdata/aws_hourly_obs/surface_aws_hourly_20210720_0900.csv
文件已存在，跳过下载：/mnt/e/metradar_test/radar_aws/awsdata/aws_hourly_obs/surface_aws_hourly_20210720_1000.csv
文件已存在，跳过下载：/mnt/e/metradar_test/radar_aws/awsdata/aws_hourly_obs/surface_aws_hourly_20210720_1100.csv
文件已存在，跳过下载：/mnt/e/metradar_test/radar_aws/awsdata/aws_hourly_obs/surface_aws_hourly_20210720_1200.csv


In [2]:
# 下载自动站分钟级观测数据
from nmc_met_io.retrieve_cmadaas import cmadaas_obs_in_rect_by_time_range
import os
from datetime import datetime, timedelta
starttime = '20210720080000'
endtime =   '20210720090000'
outpath='/mnt/e/metradar_test/radar_aws/awsdata/aws_minutes_obs'
region = [33,36,112,115]
if not os.path.exists(outpath):
    os.makedirs(outpath, exist_ok=True)
startt = datetime.strptime(starttime, '%Y%m%d%H%M%S')
endt = datetime.strptime(endtime, '%Y%m%d%H%M%S')
# 每次下载6分钟的数据
curt = startt
while curt <= endt:
    nextt = curt + timedelta(minutes=6)
    curtime = curt.strftime('%Y%m%d%H%M%S')
    nexttime = nextt.strftime('%Y%m%d%H%M%S')
    outname = f'{outpath}/surface_aws_minute_{curtime[0:8]}_{curtime[8:12]}_{nexttime[8:12]}.csv'
    if os.path.exists(outname):
        print(f'文件已存在，跳过下载：{outname}')
        curt = nextt
        continue
    # 下载数据
    repd = cmadaas_obs_in_rect_by_time_range( 
                                        time_range=f'[{curtime},{nexttime}]',
                                limit=region,data_code='SURF_CHN_MUL_MIN', trans_type=False,
                                elements="Station_Name,Province,City,Cnty,Station_Id_d,\
                                    Station_Id_C,Station_levl,Lat,Lon,Alti,Year,Mon,Day,Hour,Min,\
                                        PRS_Sea,WIN_S_INST,WIN_D_INST,WIN_S_Avg_2mi,\
                                    WIN_D_Avg_2mi,GST,TEM,RHU,PRE,DPT")
    # 去掉列名中的空格
    repd.columns = [col.strip() for col in repd.columns]
    timetag=[]
    for mbm in range(repd['Year'].shape[0]):
        timetag.append('%04d%02d%02d%02d%02d'%(int(repd['Year'].values[mbm]),int(repd['Mon'].values[mbm]),\
            int(repd['Day'].values[mbm]),int(repd['Hour'].values[mbm]),int(repd['Min'].values[mbm])))
    repd['timetag'] = timetag
    repd = repd.sort_values('timetag', ascending=False)
    #去掉重复的行，只保留一次出现的（从时间上来看最近的）
    norepeat_df = repd.drop_duplicates(subset=['Station_Id_d'], keep='first')
    # surface_aws_20250914_0930.csv
    norepeat_df.to_csv(outname, index=False,encoding='GBK')
    print(f'已保存到文件：{outname}')
    
    curt = nextt

文件已存在，跳过下载：/mnt/e/metradar_test/radar_aws/awsdata/aws_minutes_obs/surface_aws_minute_20210720_0800_0806.csv
文件已存在，跳过下载：/mnt/e/metradar_test/radar_aws/awsdata/aws_minutes_obs/surface_aws_minute_20210720_0806_0812.csv
文件已存在，跳过下载：/mnt/e/metradar_test/radar_aws/awsdata/aws_minutes_obs/surface_aws_minute_20210720_0812_0818.csv
文件已存在，跳过下载：/mnt/e/metradar_test/radar_aws/awsdata/aws_minutes_obs/surface_aws_minute_20210720_0818_0824.csv
文件已存在，跳过下载：/mnt/e/metradar_test/radar_aws/awsdata/aws_minutes_obs/surface_aws_minute_20210720_0824_0830.csv
文件已存在，跳过下载：/mnt/e/metradar_test/radar_aws/awsdata/aws_minutes_obs/surface_aws_minute_20210720_0830_0836.csv
文件已存在，跳过下载：/mnt/e/metradar_test/radar_aws/awsdata/aws_minutes_obs/surface_aws_minute_20210720_0836_0842.csv
文件已存在，跳过下载：/mnt/e/metradar_test/radar_aws/awsdata/aws_minutes_obs/surface_aws_minute_20210720_0842_0848.csv
文件已存在，跳过下载：/mnt/e/metradar_test/radar_aws/awsdata/aws_minutes_obs/surface_aws_minute_20210720_0848_0854.csv
文件已存在，跳过下载：/mnt/e/metradar_t

In [3]:
# 下载雷达单站基数据
from nmc_met_io.retrieve_cmadaas import cmadaas_radar_level2_by_timerange_and_id
outpath = '/mnt/e/metradar_test/radar_aws/radar_fmt/'
radar_ids = "Z9371"
time_range = "[20210720080000,20210720120000]"


cmadaas_radar_level2_by_timerange_and_id(radar_ids=radar_ids, time_range=time_range, outpath=outpath)

File already exists: /mnt/e/metradar_test/radar_aws/radar_fmt//2021/07/20/Z9371/Z_RADR_I_Z9371_20210720080000_O_DOR_SAD_CAP_FMT.bin.bz2
File already exists: /mnt/e/metradar_test/radar_aws/radar_fmt//2021/07/20/Z9371/Z_RADR_I_Z9371_20210720080601_O_DOR_SAD_CAP_FMT.bin.bz2
File already exists: /mnt/e/metradar_test/radar_aws/radar_fmt//2021/07/20/Z9371/Z_RADR_I_Z9371_20210720081201_O_DOR_SAD_CAP_FMT.bin.bz2
File already exists: /mnt/e/metradar_test/radar_aws/radar_fmt//2021/07/20/Z9371/Z_RADR_I_Z9371_20210720082400_O_DOR_SAD_CAP_FMT.bin.bz2
File already exists: /mnt/e/metradar_test/radar_aws/radar_fmt//2021/07/20/Z9371/Z_RADR_I_Z9371_20210720083000_O_DOR_SAD_CAP_FMT.bin.bz2
File already exists: /mnt/e/metradar_test/radar_aws/radar_fmt//2021/07/20/Z9371/Z_RADR_I_Z9371_20210720083600_O_DOR_SAD_CAP_FMT.bin.bz2
File already exists: /mnt/e/metradar_test/radar_aws/radar_fmt//2021/07/20/Z9371/Z_RADR_I_Z9371_20210720084200_O_DOR_SAD_CAP_FMT.bin.bz2
File already exists: /mnt/e/metradar_test/radar_

True

### 第二步，开始绘图

In [1]:

import pandas as pd
import numpy as np  
from metradar.graph.draw_radar_comp_func import DRAW_RADAR_OTHER,ini_params
# 准备GIS文件
gisfile = '/home/wjzhu/OneDrive/PythonCode/MyWork/metradar/resources/stations/cma_city_station_info.dat'

cities = pd.read_csv(gisfile,  delimiter=r"\s+") 

cityname = np.unique(cities['city_name'])
stanames=[]
lat=[]
lon=[]

curdata = cities
for name in cityname:
    flag2 =  curdata['city_name'] == name
    tmp_lat = np.mean(curdata[flag2]['lat'])
    tmp_lon = np.mean(curdata[flag2]['lon'])

    stanames.append(name)
    lon.append(tmp_lon)
    lat.append(tmp_lat)
    
    
# 初始化绘图参数
params=ini_params()

# 修改补充完善参数
params['radarfile_path'] = '/mnt/e/metradar_test/radar_aws/radar_fmt/2021/07/20/Z9371'
params['radarfile_name'] = 'Z_RADR_I_Z9371_20210720083000_O_DOR_SAD_CAP_FMT.bin.bz2'
params['rose_cr_path'] = '/mnt/e/metradar_test/radar_aws/pup/Z9371/CR/37'
params['rose_cr_name'] = 'Z9371_20210720083000Z_CR_00_37'
params['aws_hour_file_path'] = '/mnt/e/metradar_test/radar_aws/awsdata/aws_hourly_obs'
params['aws_hour_file_name'] = 'surface_aws_hourly_20210720_0900.csv'
params['aws_min_file_path'] = '/mnt/e/metradar_test/radar_aws/awsdata/aws_minutes_obs'
params['aws_min_file_name'] = 'surface_aws_minute_20210720_0824_0830.csv'
params['pic_path'] = '/mnt/e/metradar_test/radar_aws/pic'
params['slat'] = 34
params['nlat'] = 35
params['wlon'] = 113
params['elon'] = 114
params['timestr'] = params['radarfile_name'].split('_')[4][0:8] + '.' +  params['radarfile_name'].split('_')[4][8:14]
params['fontfile'] = '/home/wjzhu/OneDrive/PythonCode/MyWork/metradar/resources/fonts/YaHeiConsolasHybrid_1.12.ttf'
params['gis_lats'] = lat
params['gis_lons'] = lon
params['gis_name'] = stanames
params['breplace'] = True
params['bdraw_crs'] = True
params['crs_start_azi'] = 300
params['crs_start_range'] = 50 # km
params['crs_end_azi'] = 300
params['crs_end_range'] = 100 # km
params['top_height'] = 15 # km
params['ref_colorfile'] = '/home/wjzhu/OneDrive/PythonCode/MyWork/metradar/resources/gr2_colors/default_BR_PUP2.pal'
params['vel_colorfile'] = '/home/wjzhu/OneDrive/PythonCode/MyWork/metradar/resources/gr2_colors/default_BV_PUP2.pal'
params['figsize_width'] = 4
params['fontsize_gis'] = 5
params['fontsize_colorbar'] = 5
params['fontsize_title'] = 6
params['mapcolor'] = [0/255,0/255,0/255]
params['dpi'] = 300
params['pic_format'] = 'png'
params['bdraw_title_ppi'] = False
params['wind_bar_width_inner']=0.5
params['wind_bar_width_outter']=1





## You are using the Python ARM Radar Toolkit (Py-ART), an open source
## library for working with weather radar data. Py-ART is partly
## supported by the U.S. Department of Energy as part of the Atmospheric
## Radiation Measurement (ARM) Climate Research Facility, an Office of
## Science user facility.
##
## If you use this software to prepare a publication, please cite:
##
##     JJ Helmus and SM Collis, JORS 2016, doi: 10.5334/jors.119



In [2]:
# 创建绘图函数
_draw_radar_other = DRAW_RADAR_OTHER(params)

_draw_radar_other.read_vol_data()

True

In [3]:
_draw_radar_other.get_crs_from_radar()
_draw_radar_other.draw_crs(varname='reflectivity',subdir='垂直剖面',brefresh=True)

reflectivity
cross_correlation_ratio
differential_phase
spectrum_width
differential_reflectivity
velocity
/mnt/e/metradar_test/radar_aws/pic/垂直剖面/20210720.083000.reflectivity.png


In [10]:

_draw_radar_other.draw_ref_alone(subdir='ref',tilt=0,thred=-5)


image saved: /mnt/e/metradar_test/radar_aws/pic/0.5_ref/20210720.083000.png


In [11]:
_draw_radar_other.draw_ref_pre(subdir='ref_pre',)

image saved: /mnt/e/metradar_test/radar_aws/pic/ref_pre/20210720.083000.png


In [3]:
_draw_radar_other.draw_ref_wind_barb_vortex(subdir='ref_pre_wind_vortex',)

image saved: /mnt/e/metradar_test/radar_aws/pic/ref_pre_wind_vortex/20210720.083000.png


In [6]:
_draw_radar_other.draw_vel_wind_quiver(subdir='vel_wind_quiver',tilt=4)

image saved: /mnt/e/metradar_test/radar_aws/pic/vel_wind_quiver/20210720.083000.png


In [5]:
_draw_radar_other.draw_vel_alone(subdir='vel',tilt=4)

image saved: /mnt/e/metradar_test/radar_aws/pic/2.5_vel/20210720.083000.png


In [12]:
# xlimits和ylimits参数范围不要设置的过大，否则运行会很慢，建议在-50到50之间
_draw_radar_other.get_cref_from_radar(xlimits=[-80,50],ylimits=[-80,50])


enter get_cref_from_radar function!
grid shape: (29, 131, 131)
grid limits: ((1000, 15000), (-80000, 50000), (-80000, 50000))
grid origin: (np.float64(34.704444885253906), np.float64(113.69721984863281))


In [2]:
# 直接从ROSE产品中读取组合反射率因子数据
_draw_radar_other = DRAW_RADAR_OTHER(params)
_draw_radar_other.get_cref_from_rose()


产品类型: 18-CR
数据类型: 2-滤波后反射率(Reflectivity)


True

In [3]:
# 以组合反射率为基础绘图

_draw_radar_other.draw_cref_wind_barb(subdir='cref_wind_barb',)


/mnt/e/metradar_test/radar_aws/pic/cref_wind_barb/20210720.083000.png


In [ ]:
_draw_radar_other.draw_cref_pre(subdir='cref_pre',)
_draw_radar_other.draw_cref_wind_barb(subdir='cref_wind_barb',)
_draw_radar_other.draw_cref_wind_quiver(subdir='cref_wind_quiver',)
_draw_radar_other.draw_cref_pre_wind_barb(subdir='cref_pre_wind_barb',)